## **Fake news prediction**

Now it's time to apply some ML algorithms to our data and try to make a model which can distinguish between fake and real news articles

In [69]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")

In [2]:
news_df = pd.read_csv('news_df_cleaned.csv')

In [3]:
fake_df = news_df[news_df['classification'] == 'fake']
real_df = news_df[news_df['classification'] == 'real']

Convert the clasification column to an integer so the models can process it

In [4]:
fake_df["class"] = 0
real_df["class"] = 1

Merging the two dfs into one df again

In [5]:
df_merged = pd.concat([fake_df, real_df], axis =0 )
df_merged

,headline,source,text,classification,text_cleaned,text_tokenized,text_removed_stopwords,text_lemmatized,headline_cleaned,headline_tokenized,headline_removed_stopwords,headline_lemmatized,class
7262,Wikileaks: Podesta Had Disturbing ‘Beyond Disc...,Baxter Dmitry,"by Baxter Dmitry in News , US // 0 Comments ...",fake,by baxter dmitry in news us 0 comments a new ...,"['by', 'baxter', 'dmitry', 'in', 'news', 'us',...","['baxter', 'dmitry', 'news', 'us', '0', 'comme...","['baxter', 'dmitry', 'news', 'u', '0', 'commen...",wikileaks podesta had disturbing beyond discip...,"['wikileaks', 'podesta', 'had', 'disturbing', ...","['wikileaks', 'podesta', 'disturbing', 'beyond...","['wikileaks', 'podesta', 'disturbing', 'beyond...",0
7263,Trump Elected As President – Risks And Opportu...,The Saker,"13904 Views November 09, 2016 193 Comments Ana...",fake,13904 views november 09 2016 193 comments anal...,"['13904', 'views', 'november', '09', '2016', '...","['13904', 'views', 'november', '09', '2016', '...","['13904', 'view', 'november', '09', '2016', '1...",trump elected as president risks and opportuni...,"['trump', 'elected', 'as', 'president', 'risks...","['trump', 'elected', 'president', 'risks', 'op...","['trump', 'elected', 'president', 'risk', 'opp...",0
7264,Break The Silence Or Support Self-Determinatio...,shorty,B y Danny Haiphong S yria is “the target of on...,fake,b y danny haiphong s yria is the target of one...,"['b', 'y', 'danny', 'haiphong', 's', 'yria', '...","['b', 'danny', 'haiphong', 'yria', 'target', '...","['b', 'danny', 'haiphong', 'yria', 'target', '...",break the silence or support self determinatio...,"['break', 'the', 'silence', 'or', 'support', '...","['break', 'silence', 'support', 'self', 'deter...","['break', 'silence', 'support', 'self', 'deter...",0
7265,Re: Donald-Trump-Accepts-The-Nomination-Public...,Anton Ossa,Wilderness Survival Copyright © 2016 The Econo...,fake,wilderness survival copyright 2016 the economi...,"['wilderness', 'survival', 'copyright', '2016'...","['wilderness', 'survival', 'copyright', '2016'...","['wilderness', 'survival', 'copyright', '2016'...",re donald trump accepts the nomination public ...,"['re', 'donald', 'trump', 'accepts', 'the', 'n...","['donald', 'trump', 'accepts', 'nomination', '...","['donald', 'trump', 'accepts', 'nomination', '...",0
7266,Re: More Than 101 Million Working Age American...,peggie gor,More Than 101 Million Working Age Americans Do...,fake,more than 101 million working age americans do...,"['more', 'than', '101', 'million', 'working', ...","['101', 'million', 'working', 'age', 'american...","['101', 'million', 'working', 'age', 'american...",re more than 101 million working age americans...,"['re', 'more', 'than', '101', 'million', 'work...","['101', 'million', 'working', 'age', 'american...","['101', 'million', 'working', 'age', 'american...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7257,2020 Census Results Show Big Changes To Americ...,AllSides Headline Roundup,"According to the 2020 United States Census, th...",real,according to the 2020 united states census the...,"['according', 'to', 'the', '2020', 'united', '...","['according', '2020', 'united', 'states', 'cen...","['according', '2020', 'united', 'state', 'cens...",2020 census results show big changes to americ...,"['2020', 'census', 'results', 'show', 'big', '...","['2020', 'census', 'results', 'show', 'big', '...","['2020', 'census', 'result', 'show', 'big', 'c...",1
7258,"Dems And Gop Strike Short-Term Deal On Debt, B...",AllSides Headline Roundup,Democrats and Republicans have reached a tempo...,real,democrats and republicans have reached a tempo...,"['democrats', 'and', 'republicans', 'have', 'r...","['democrats', 'republicans', 'reached', 'tempo...","['democrat', 'republican', 'reached', 'tempora...",dems and gop strike short term deal on debt bu...,"['dems', 'and', 'gop', 'strike', 'short', 'ter...","['dems', 'gop', 'strike', 'short', 'term', 'de...","['dems', 'gop', 'strike', 

Dropping the unecessary columns

In [6]:
df_merged = df_merged.drop(columns=['headline', 'source', 'text', 'classification', 
'text_cleaned', 'text_tokenized', 'text_removed_stopwords', 'headline_cleaned', 'headline_tokenized', 'headline_removed_stopwords'])

Randomly shuffle df and reset the index

In [7]:
df_merged = df_merged.sample(frac = 1)
df_merged.head()


,text_lemmatized,headline_lemmatized,class
7753,"['home', 'headline', 'world', 'news', 'fix', '...","['fix', 'nbc', 'affiliate', 'accidentally', 'p...",0
3824,"['syria', 'opposition', 'rejected', 'russiansp...","['syrian', 'opposition', 'reject', 'russia', '...",1
1911,"['man', 'forecast', 'replace', 'cuban', 'presi...","['likely', 'successor', 'cuba', 'castro', 'rej...",1
10713,"['bni', 'store', 'oct', '27', '2016', 'germany...","['germany', 'gang', 'syrian', 'muslim', 'boy',...",0
12203,"['know', 'various', 'governmental', 'instituti...","['breaking', 'official', 'china', 'government'...",0


In [8]:
df_merged.reset_index(inplace = True)
df_merged.drop(["index"], axis = 1, inplace = True)

In [9]:
df_merged.head()


,text_lemmatized,headline_lemmatized,class
0,"['home', 'headline', 'world', 'news', 'fix', '...","['fix', 'nbc', 'affiliate', 'accidentally', 'p...",0
1,"['syria', 'opposition', 'rejected', 'russiansp...","['syrian', 'opposition', 'reject', 'russia', '...",1
2,"['man', 'forecast', 'replace', 'cuban', 'presi...","['likely', 'successor', 'cuba', 'castro', 'rej...",1
3,"['bni', 'store', 'oct', '27', '2016', 'germany...","['germany', 'gang', 'syrian', 'muslim', 'boy',...",0
4,"['know', 'various', 'governmental', 'instituti...","['breaking', 'official', 'china', 'government'...",0


### X/y split

In [10]:
X = df_merged['text_lemmatized']
y = df_merged['class']

Function to test some different ml algorithms
- we will try logistic regression, decision tree & random forest

In [94]:
def model_tester(X, y, stopwords=None):
    
    # test/train split

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

    # vectorization: this maps the text to a corresponding vector of real numbers so that it can be processed by a ML algorithm

    vectorization = TfidfVectorizer(stop_words=stopwords)
    xv_train = vectorization.fit_transform(X_train)
    xv_test = vectorization.transform(X_test)

    # here are the parameters to test

    log_reg_params = [{"C":0.01}, {"C":0.1}, {"C":1}, {"C":10}]
    dec_tree_params = [{"criterion": "gini"}, {"criterion": "entropy"}]
    rand_for_params = [{"criterion": "gini"}, {"criterion": "entropy"}]

    modelclasses = [
        ["log regression", LogisticRegression, log_reg_params],
        ["decision tree", DecisionTreeClassifier, dec_tree_params],
        ["random forest", RandomForestClassifier, rand_for_params]
    ]

    # try out each model with the parameters give above and append to a list
    
    insights = []
    for modelname, Model, params_list in modelclasses:
        for params in params_list:
            model = Model(**params)
            model.fit(xv_train, y_train)
            score = model.score(xv_test, y_test)
            preds = model.predict(xv_test)
            f1score = f1_score(y_test, preds)
            cv_score = cross_val_score(model, xv_train, y_train, cv=5).mean()
            insights.append((modelname, model, params, score, f1score, cv_score))

    insights.sort(key=lambda x:x[-1], reverse=True)
    for modelname, model, params, cv_score, score, f1score in insights:
        print(modelname, params, 'cv_score:', round(cv_score, 4), 'accuracy_score:', round(score, 4), 'f1_score:', round(f1score, 4))

In [95]:
model_tester(X, y)

log regression {'C': 10} cv_score: 0.9646 accuracy_score: 0.9636 f1_score: 0.9589
random forest {'criterion': 'entropy'} cv_score: 0.9467 accuracy_score: 0.9459 f1_score: 0.9455
log regression {'C': 1} cv_score: 0.9499 accuracy_score: 0.9491 f1_score: 0.9455
random forest {'criterion': 'gini'} cv_score: 0.9413 accuracy_score: 0.9402 f1_score: 0.9411
log regression {'C': 0.1} cv_score: 0.928 accuracy_score: 0.9274 f1_score: 0.922
log regression {'C': 0.01} cv_score: 0.9029 accuracy_score: 0.9014 f1_score: 0.9038
decision tree {'criterion': 'entropy'} cv_score: 0.8948 accuracy_score: 0.8944 f1_score: 0.9012
decision tree {'criterion': 'gini'} cv_score: 0.8977 accuracy_score: 0.8963 f1_score: 0.8999


Function to find out which features are most important in the random forest model

- the scores are high so we could have overfitting

In [97]:
def find_imp_features(X, y, Model):

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

    vectorization = TfidfVectorizer()
    xv_train = vectorization.fit_transform(X_train)
    xv_test = vectorization.transform(X_test)

    model = Model

    model.fit(xv_train, y_train)

    new_train_df = pd.DataFrame()
    new_test_df = pd.DataFrame()

    for i, col in enumerate(vectorization.get_feature_names()):
        new_train_df[col] = pd.arrays.SparseArray(xv_train[:, i].toarray().ravel(), fill_value=0)
    
    for i, col in enumerate(vectorization.get_feature_names()):
        new_test_df[col] = pd.arrays.SparseArray(xv_test[:, i].toarray().ravel(), fill_value=0) 

    importances = model.feature_importances_
    std = np.std([tree.feature_importances_ for tree in model.estimators_],
             axis=0)
    indices = np.argsort(importances)[::-1]

    # Print the feature ranking
    print("Feature ranking:")

    for f in range(0,20): 
        print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]),new_train_df.iloc[:,indices[f]].name)

    number_features_print = 10
    height = [importances[indices[i]] for i in range(0,number_features_print)]
    bars = [new_train_df.iloc[:,indices[x]].name for x in range(0,number_features_print)]


    height.reverse()
    bars.reverse()

    y_pos = np.arange(len(bars))
 
    #   Create horizontal bars
    plt.barh(y_pos, height)
 
    # Create names on the y-axis
    plt.yticks(y_pos, bars)
    plt.title('Top predictors',fontsize=19)
    plt.yticks(fontsize=13)
    plt.xticks(fontsize=13)
    plt.ylabel('Word', fontsize = 16)
    plt.xlabel('Random Forest Coefficient', fontsize = 16)
    plt.figure(figsize=(13, 13))

    # Show graphic
    plt.show();

In [99]:
find_imp_features(X, y, RandomForestClassifier(criterion= 'entropy'))

NotFittedError: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [83]:
model_tester(X, y, stopwords=['2016', 'said', 'thursday', 'october', 'friday', 'know'])

log regression {'C': 10} cv_score: 0.9501 accuracy_score: 0.9491 f1_score: 0.9507
log regression {'C': 1} cv_score: 0.9415 accuracy_score: 0.9406 f1_score: 0.9388
random forest {'criterion': 'gini'} cv_score: 0.9354 accuracy_score: 0.9346 f1_score: 0.9309
random forest {'criterion': 'entropy'} cv_score: 0.9336 accuracy_score: 0.9328 f1_score: 0.9297
log regression {'C': 0.1} cv_score: 0.9147 accuracy_score: 0.9152 f1_score: 0.9113
log regression {'C': 0.01} cv_score: 0.8923 accuracy_score: 0.8927 f1_score: 0.8907
decision tree {'criterion': 'gini'} cv_score: 0.8636 accuracy_score: 0.8608 f1_score: 0.8646
decision tree {'criterion': 'entropy'} cv_score: 0.8582 accuracy_score: 0.8559 f1_score: 0.8615


Function to print confusion matrix


In [ ]:
sns.set_style('white')
plot_confusion_matrix(model, xv_test, y_test, cmap='Blues')
plt.show()